# Algoritmo para geração do Dataset baseado na energia do Hassel e nos autovalores do Kapetanovic

## Importações

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import sys
import pandas as pd
from joblib import Parallel, delayed
from time import time
from IPython.display import display, clear_output

## Definição de funções

Vou copiar as funções do arquivo comm.py aqui pra garantir que se mudar lá não vai mudar aqui, e tb pq vou fazer algumas modificações na função da matriz R. 

In [2]:
def bin2gray(bi):
    order = bi.shape[1]
    lin = bi.shape[0]
    g = np.zeros((lin, order))

    idx = 0
    for bits in bi:
        g[idx, 0] = int(bits[0])
        for l in range(1, order):
            g[idx, l] = int(bits[l] ^ bits[l - 1])
        idx = idx + 1

    return g

In [3]:
def qpskmodulator(input_bits, average_power=1, phase_offset=np.pi / 4):

    # Generate symbols
    symbols = np.array([1, 1j, -1, -1j]) * np.exp(1j * phase_offset)
    symbols = np.sqrt(average_power) * symbols

    input_arr = input_bits.reshape((int(len(input_bits) / 2)), 2)
    gray_code = bin2gray(input_arr)

    output = [
    ]  # Pre allocate in the future to prevent unnecessary memory usage
    for symbol in gray_code:
        #         print(symbol)
        s = int(2 * symbol[0] + symbol[1])
        output.append(symbols[s])

    return np.array(output)

In [4]:
def awgn(sig, SNR, sig_power=1):
    # SNR in dB

    SNR_linear = 10**(SNR / 10)  #SNR to linear scale
    noise_power = sig_power / SNR_linear
    # Noise power

    dim = sig.shape
    if not np.isreal(sig[1, 1]):
        noise = np.sqrt(noise_power / 2) * (np.random.normal(0, 1, dim) +
                                            1j * np.random.normal(0, 1, dim))
    else:
        noise = np.sqrt(noise_power) * np.random.normal(0, 1, dim)

    sig_noisy = sig + noise

    return sig_noisy

In [5]:
def razaoMatrizR(Y, qtdSimbolosPiloto, qtdAntenas, snr, potenciaSinal=1):
    
    # OBTENDO A POTENCIA DO RUIDO N0
    N0 = potenciaSinal/(10**(snr/10))
    
    # CRIANDO A MATRIZ IDENTIDADE E FAZENDO A OPERACAO HERMITIANA EM Y
    I  = np.identity(qtdSimbolosPiloto)
    YH = np.conjugate(Y).T
    
    # CONSTRUINDO A MATRIZ R
    R = np.matmul(YH, Y)/qtdAntenas - N0 * I
    
    # DOIS MAIORES AUTOVALORES
    autovalores = np.sort(np.linalg.eigvals(R))
    a1 = autovalores[-1]
    a2 = autovalores[-2]
    a3 = autovalores[-3]
    
    return a1.real, a2.real, a3.real

In [6]:
def simularPropagacaoSinal(qtdSimbolosPiloto, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes=1):
    
    # SIMULANDO OS CANAIS DO USUARIO E DO ESPIAO
    Haut = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)) + 1j*np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)))
    g    = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdEspioes)) + 1j*np.random.normal(0, 1, (qtdAntenas, qtdEspioes)))

    # SEQUENCIA PILOTO ALEATORIA + MODULACAO PARA TODOS OS USUARIOS:
    bitStream = np.random.choice([0, 1], qtdSimbolosPiloto*bitsPorSimbolo*qtdUsuarios)
    symb      = qpskmodulator(bitStream) # QPSK Modulator
    xp        = symb.reshape(qtdUsuarios, qtdSimbolosPiloto)

    # ESPIAO ENTRANDO NA JOGADA (SE A POTENCIA DELE FOR 0 ELE NAO ENTRA NA JOGADA):
    xpe  = np.sqrt(potenciaEspiao)*xp[0, :] # xpe vai ser a sequencia piloto do primeiro usuario multiplicada pela raiz da potencia do espiao
    xptx = np.concatenate((xp, [xpe])) # xptx sera a matriz xp com uma linha a mais: xpe
    H    = np.concatenate((Haut, g), axis=1) # H vai ser Haut com uma COLUNA a mais, que vai ser g

    # TRANSMISSAO PELO CANAL
    Y = np.dot(H, xptx) # fading
    Y = awgn(Y, SNR=snr) # ruido branco
    
    return Y, xp, H[:,0]

In [7]:
def estimarCanal(Y, xp):
    # HEST =(Y * xptranspostoconjugado ) * ((xp*xptranspconj)^(-1))
    # HEST -> cada coluna um usuario. No caso de um único usuário pegar a primeira coluna
    # return Hest(coluna 0)
    
    Hest = np.matmul(np.matmul(Y, np.conjugate(xp).T), np.linalg.lstsq(np.matmul(xp, np.conjugate(xp).T), np.eye(np.matmul(xp, np.conjugate(xp).T).shape[0], np.matmul(xp, np.conjugate(xp).T).shape[0]))[0])
    
    return Hest[:,0]

In [8]:
def calcularEnergiaHassan(HestCol0, snr, qtdAntenas, qtdSimbolosPiloto):
    
    N0       = 1/(10**(snr/10))
    sovertau = qtdAntenas*N0/qtdSimbolosPiloto
    ln       = np.log((2+sovertau)/(1+sovertau))
    eta      = (1 + sovertau)*(2+sovertau)*ln
    E        = np.matmul(np.conjugate(HestCol0).T, HestCol0)/qtdAntenas
    
    return E.real, eta.real

In [9]:
def obterPrincipaisFeatures(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes):
    
    # PROPAGACAO DO SINAL
    Y, xp, HCol0 = simularPropagacaoSinal(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes)
        
    # CALCULANDO A RAZAO DOS AUTOVALORES (kapetanovik)
    a1, a2, a3 = razaoMatrizR(Y, qtdSimbolos, qtdAntenas, snr)    
    
    # ESTIMATIVA DO CANAL (hassel)
    HestCol0 = estimarCanal(Y, xp)
    
    # CALCULO DA ENERGIA (hassel)
    E, eta = calcularEnergiaHassan(HestCol0, snr, qtdAntenas, qtdSimbolos)
    
    return E, eta, a1, a2, a3

In [10]:
def main(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes, caminhoCSV):
    
    # OBTENDO AS PRINCIPAIS FEATURES
    E, eta, a1, a2, a3 = obterPrincipaisFeatures(qtdSimbolos, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes)

    # FAZENDO O TARGET DO DATASET
    ataquePresente = 0
    if potenciaEspiao > 0:
        ataquePresente = 1

    # SALVANDO NO CSV - mode='a' significa append
    linhaNovaCSV = pd.DataFrame([[
        qtdAntenas, #qtdUsuarios, #snr, 
        E,
        eta,
        a1,
        a2,
        a3,
        potenciaEspiao, 
        ataquePresente
    ]]).to_csv(caminhoCSV, mode='a', header=False, index=False)

In [11]:
def criarDatasetTeste0(qtdUsuarios, qtdAntenas, qtdSimbolos, rangePotenciaEspiao, rangeSNRs, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas):
    # INICIANDO O CSV
    caminhoCSV = "../../CSV/train_0_influencia_snr/test_dataset_"+str(qtdUsuarios)+"usuarios_"+str(qtdAntenas)+"antenas_"+str(qtdSimbolos)+"simbolos_"+str(time())+".csv"
    dataframe   = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)

    # PARA PRINTAR O PROGRESSO
    iteracaoAtual     = 1 
    qtdPossibilidades = len(rangePotenciaEspiao) * len(rangeSNRs)

    # ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS
    for potenciaEspiaoAtual in rangePotenciaEspiao:
        for snrAtual in rangeSNRs:

            # VAMOS PARALELIZAR PQ NEM DEUS TEM A PACIENCIA NECESSARIA KKKKK
            Parallel(n_jobs=nJobs, verbose=0)(delayed(main)(qtdSimbolos, qtdBitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiaoAtual, snrAtual, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))            
            
            # PRINT
            clear_output(wait=True)
            display("Usuários: "+str(qtdUsuarios)+". Antenas: "+str(qtdAntenas)+". Símbolos: "+str(qtdSimbolos)+". Progresso: "+str(100*(iteracaoAtual/qtdPossibilidades))[:7]+"%.")
            iteracaoAtual += 1

In [12]:
def criarDatasetTeste1(snr, qtdAntenas, qtdSimbolos, rangePotenciaEspiao, rangeQtdUsuarios, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas):
    # INICIANDO O CSV
    caminhoCSV = "../../CSV/teste_1_influencia_qtd_usuarios/train_dataset_"+str(snr)+"dB_"+str(qtdAntenas)+"antenas_"+str(qtdSimbolos)+"simbolos_"+str(time())+".csv"
    dataframe   = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)

    # PARA PRINTAR O PROGRESSO
    iteracaoAtual     = 1 
    qtdPossibilidades = len(rangePotenciaEspiao) * len(rangeQtdUsuarios)

    # ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS
    for potenciaEspiaoAtual in rangePotenciaEspiao:
        for qtdUsuarios in rangeQtdUsuarios:

            # VAMOS PARALELIZAR PQ NEM DEUS TEM A PACIENCIA NECESSARIA KKKKK
            Parallel(n_jobs=nJobs, verbose=0)(delayed(main)(qtdSimbolos, qtdBitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiaoAtual, snr, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))            
            
            # PRINT
            clear_output(wait=True)
            display("SNR: "+str(snr)+". Antenas: "+str(qtdAntenas)+". Símbolos: "+str(qtdSimbolos)+". Progresso: "+str(100*(iteracaoAtual/qtdPossibilidades))[:7]+"%.")
            iteracaoAtual += 1

In [13]:
def criarDatasetTeste2(snr, rangeQtdAntenas, qtdSimbolos, rangePotenciaEspiao, qtdUsuarios, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas):
    # INICIANDO O CSV
    caminhoCSV  = "../../CSV/teste_2_influencia_qtd_antenas/test_dataset_"+str(snr)+"dB_"+str(qtdUsuarios)+"usuarios_"+str(qtdSimbolos)+"simbolos_"+str(time())+".csv"
    dataframe   = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)

    # PARA PRINTAR O PROGRESSO
    iteracaoAtual     = 1 
    qtdPossibilidades = len(rangePotenciaEspiao) * len(rangeQtdAntenas)

    # ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS
    for potenciaEspiaoAtual in rangePotenciaEspiao:
        for qtdAntenas in rangeQtdAntenas:

            # VAMOS PARALELIZAR PQ NEM DEUS TEM A PACIENCIA NECESSARIA KKKKK
            Parallel(n_jobs=nJobs, verbose=0)(delayed(main)(qtdSimbolos, qtdBitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiaoAtual, snr, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))            
            
            # PRINT
            clear_output(wait=True)
            display("SNR: "+str(snr)+". Usuários: "+str(qtdUsuarios)+". Símbolos: "+str(qtdSimbolos)+". Progresso: "+str(100*(iteracaoAtual/qtdPossibilidades))[:7]+"%.")
            iteracaoAtual += 1

## Parâmetros Iniciais

In [14]:
# PARAMETROS FIXOS
nJobs             = 3
qtdEspioes        = 1
qtdBitsPorSimbolo = 2
repetibilidade    = 1000
potenciaUsuario   = 1

# PARAMETROS DOS CSVS
colunas = [
    "qtdAntenas", #"qtdUsuarios", #"snr",
    "E",
    "eta",
    "a1",
    "a2",
    "a3",
    "potenciaEspiao", 
    "ataquePresente"
]

# PARAMETROS VARIAVEIS
rangePotenciaEspiao = np.arange(0.0, 2.51, 0.5)
rangeSNRs           = np.array([10])
rangeQtdUsuarios    = np.array([64])
rangeQtdAntenas     = np.arange(64, 257, 8)
rangeQtdSimbolos    = np.array([128])

# # TESTES RAPIDOS - DEIXAR COMENTADOOOOOOO
# repetibilidade      = 10                    # TESTES RAPIDOS
# rangePotenciaEspiao = np.array([0, 1, 2])   # TESTES RAPIDOS
# rangeSNRs           = np.arange(-10, 31, 5) # TESTES RAPIDOS 
# rangeQtdUsuarios    = np.array([1])         # TESTES RAPIDOS 
# rangeQtdAntenas     = np.array([256])       # TESTES RAPIDOS 
# rangeQtdSimbolos    = np.array([128])       # TESTES RAPIDOS 

## Balanceamento do Dataset

Queremos que a quantidade de cenários em que há ataque seja igual à quantidade de casos em que não há. Se o array de potências do espião for [0.0, 0.5, 1.0, 1.5, 2.0, 2.5], teremos 5x mais dados em que há contaminação. Nesse caso, temos que fazer o array se transformar em [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5]. SE RODAR A CÉLULA ABAIXO MAIS DE UMA VEZ ELE VAI FICAR ADICIONANDO ZEROS. CUIDADO!

In [15]:
rangePotenciaEspiao = np.concatenate((np.zeros(len(rangePotenciaEspiao - 1) - 1), rangePotenciaEspiao[1:]))
print(rangePotenciaEspiao)

[0.  0.  0.  0.  0.  0.5 1.  1.5 2.  2.5]


## Criando os Datasets

Vamos criar um dataset para cada combinação de qtd de usuários, antenas e símbolos.

In [16]:
# TESTE_0_INFLUENCIA_DA_SNR
# for qtdUsuarios in rangeQtdUsuarios:
#     for qtdAntenas in rangeQtdAntenas:
#         for qtdSimbolos in rangeQtdSimbolos:
#             criarDatasetTeste0(qtdUsuarios, qtdAntenas, qtdSimbolos, rangePotenciaEspiao, rangeSNRs, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas)

In [17]:
# TESTE_1_INFLUENCIA_DA_QTD_USUARIOS
# for snr in rangeSNRs:
#     for qtdAntenas in rangeQtdAntenas:
#         for qtdSimbolos in rangeQtdSimbolos:
#             criarDatasetTeste1(snr, qtdAntenas, qtdSimbolos, rangePotenciaEspiao, rangeQtdUsuarios, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas)

In [18]:
# TESTE_2_INFLUENCIA_DA_QTD_ANTENAS
for snr in rangeSNRs:
    for qtdUsuarios in rangeQtdUsuarios:
        for qtdSimbolos in rangeQtdSimbolos:
            criarDatasetTeste2(snr, rangeQtdAntenas, qtdSimbolos, rangePotenciaEspiao, qtdUsuarios, qtdBitsPorSimbolo, qtdEspioes, repetibilidade, colunas)

'SNR: 10. Usuários: 64. Símbolos: 128. Progresso: 100.0%.'